# Proyecto Ciencia de Datos 2 – Ventas (UY)

**Enriquecimiento con datos públicos (INE/CKAN + SNIG)**  
**Fecha:** 2025-10-23

Este notebook está optimizado para **Google Colab**: primero instala dependencias, luego importa librerías,
descarga fuentes públicas y enriquece el dataset de ventas con:

- `nivel_desarrollo_regional` (población + empresas por departamento como proxy de actividad económica).
- `intensidad_mercado` = ventas registradas / población del departamento.

Incluye EDA (mapa coroplético, correlaciones, dispersión) y exporta un CSV enriquecido.



## Abstracto (motivación y audiencia)
Explicar diferencias de ventas entre departamentos de Uruguay incorporando contexto regional
(**población** e **indicador de actividad económica** por departamento) desde fuentes públicas (INE/CKAN y SNIG).
**Audiencia:** ejecutivos y analistas de negocio para orientar mix de productos, pricing, promociones y expansión.



## Preguntas / Hipótesis
1. ¿Las ventas son **proporcionales a la población** del departamento?
2. ¿Los departamentos con **mayor actividad económica** (proxy: más empresas) muestran **más ventas**?
3. ¿Qué departamentos presentan **mayor intensidad de mercado** (ventas per cápita)?
4. ¿Cómo se relaciona un índice simple de **desarrollo regional** (z-score de población y empresas) con las ventas?


## 1) Instalar dependencias (ejecutar UNA vez por sesión de Colab)

In [ ]:

# Si da error por versiones, volver a ejecutar esta celda y luego las importaciones.
!pip install -q pandas numpy requests geopandas pyproj shapely mapclassify matplotlib plotly openpyxl scipy


## 2) Importar librerías

In [ ]:

import pandas as pd
import numpy as np
import requests
import io
import json
import matplotlib.pyplot as plt

import geopandas as gpd
import mapclassify as mc

# Ajustes gráficos
plt.rcParams["figure.figsize"] = (7,5)
plt.rcParams["axes.grid"] = True



## 3) Fuentes públicas (APIs/descargas)
- **Ventas (tu repo):** `https://raw.githubusercontent.com/Gonrampsn4/Proyecto-Ciencia-de-Datos2-Ventas_Ramallo/main/dataset_ventas_3000.csv`
- **Población por departamento (CKAN Uruguay, CSV – “Datos básicos de Juntas Departamentales”)**  
  `https://catalogodatos.gub.uy/dataset/7e7c97c8-a7cc-4f1f-9c85-a2c25ae28141/resource/5e1cf37b-201e-43b7-a5ba-66ee0e64e4d0/download/datosbasicosjds.csv`
- **Empresas por departamento (proxy actividad – Excel público)**  
  `https://catalogodatos.gub.uy/dataset/575ccb87-ae74-4dcd-ba4b-cf050bd8e08a/resource/e8e6f2e4-357e-4027-b91a-407b5d5501f7/download/empresasdei_20230330.xlsx`
- **Geometría de departamentos (ArcGIS SNIG, GeoJSON)**  
  `https://web.snig.gub.uy/arcgisserver/rest/services/Uruguay/SNIG_Catastro/MapServer/2/query?where=1%3D1&outFields=Nombre&outSR=4326&f=geojson`


## 4) Descarga de datos

In [ ]:

ventas_url = "https://raw.githubusercontent.com/Gonrampsn4/Proyecto-Ciencia-de-Datos2-Ventas_Ramallo/main/dataset_ventas_3000.csv"
poblacion_url = "https://catalogodatos.gub.uy/dataset/7e7c97c8-a7cc-4f1f-9c85-a2c25ae28141/resource/5e1cf37b-201e-43b7-a5ba-66ee0e64e4d0/download/datosbasicosjds.csv"
empresas_xlsx_url = "https://catalogodatos.gub.uy/dataset/575ccb87-ae74-4dcd-ba4b-cf050bd8e08a/resource/e8e6f2e4-357e-4027-b91a-407b5d5501f7/download/empresasdei_20230330.xlsx"
departamentos_geojson_url = "https://web.snig.gub.uy/arcgisserver/rest/services/Uruguay/SNIG_Catastro/MapServer/2/query?where=1%3D1&outFields=Nombre&outSR=4326&f=geojson"

ventas = pd.read_csv(ventas_url)
poblacion = pd.read_csv(poblacion_url)

emp_resp = requests.get(empresas_xlsx_url)
emp_resp.raise_for_status()
empresas = pd.read_excel(io.BytesIO(emp_resp.content))

geojson = requests.get(departamentos_geojson_url).json()
gdf_deptos = gpd.GeoDataFrame.from_features(geojson["features"], crs="EPSG:4326").rename(columns={"Nombre":"Departamento"})

print("Ventas:", ventas.shape)
print("Población (raw):", poblacion.shape)
print("Empresas (raw):", empresas.shape)
print("GDF deptos:", gdf_deptos.shape)

display(ventas.head(3))
display(poblacion.head(3))
display(empresas.head(3))
gdf_deptos.head(3)


## 5) Limpieza y estandarización de Departamentos

In [ ]:

def norm_depto(s):
    if pd.isna(s): return s
    s = str(s).strip().upper()
    repl = {
        "SAN JOS\u00C9": "SAN JOSÉ",
        "SAN JOSE": "SAN JOSÉ",
        "RIO NEGRO": "RÍO NEGRO",
        "R\u00CDO NEGRO": "RÍO NEGRO",
        "PAYSANDU": "PAYSANDÚ",
        "PAYSAND\u00DA": "PAYSANDÚ",
        "TACUAREMBO": "TACUAREMBÓ",
        "TACUAREMB\u00D3": "TACUAREMBÓ",
    }
    return repl.get(s, s)

# Detectar columna de departamento en ventas
dept_cols = [c for c in ventas.columns if c.lower() in ["departamento","depto","region","provincia","estado"]]
if not dept_cols:
    raise ValueError("No se encontró columna de departamento en ventas. Renombra o crea 'Departamento'.")
ventas["Departamento"] = ventas[dept_cols[0]].apply(norm_depto)

# Normalizar población
pop_dept_cols = [c for c in poblacion.columns if "depar" in c.lower() or c.lower()=="departamento"]
if not pop_dept_cols:
    raise ValueError("CSV de población no posee columna de departamento.")
poblacion["Departamento"] = poblacion[pop_dept_cols[0]].apply(norm_depto)

# Identificar columna de población
cand_pop = [c for c in poblacion.columns if "poblac" in c.lower() or "habit" in c.lower()]
if not cand_pop:
    print("Columnas disponibles en 'poblacion':", list(poblacion.columns))
    raise ValueError("Indicar manualmente la columna de población en el CSV de población.")
poblacion = poblacion[["Departamento", cand_pop[0]]].rename(columns={cand_pop[0]:"Poblacion"})

# Empresas: agrupar por departamento
emp_dept_cols = [c for c in empresas.columns if "depar" in c.lower() or c.lower()=="departamento"]
if not emp_dept_cols:
    print("Columnas de 'empresas':", list(empresas.columns))
    raise ValueError("Excel de empresas no posee columna de departamento.")
empresas["Departamento"] = empresas[emp_dept_cols[0]].apply(norm_depto)
empresas_por_depto = empresas.groupby("Departamento", as_index=False).size().rename(columns={"size":"Empresas"})

contexto_depto = (poblacion.merge(empresas_por_depto, on="Departamento", how="left"))
contexto_depto["Empresas"] = contexto_depto["Empresas"].fillna(0).astype(int)
contexto_depto.head()


## 6) Agregación de ventas y nuevas métricas

In [ ]:

# Detectar columna de monto/importe
val_cols = [c for c in ventas.columns if any(k in c.lower() for k in ["monto","importe","total","venta","revenue","price","amount"])]
if not val_cols:
    raise ValueError("No se encontró columna de monto/importe en ventas.")
monto_col = val_cols[0]
print("Usando columna de venta:", monto_col)

ventas_depto = (ventas
                .groupby("Departamento", as_index=False)
                .agg(ventas_total=(monto_col,"sum"),
                     operaciones=("Departamento","count"),
                     ticket_promedio=(monto_col,"mean")))

panel = ventas_depto.merge(contexto_depto, on="Departamento", how="left")

# Nuevas columnas
panel["intensidad_mercado"] = panel["ventas_total"] / panel["Poblacion"]

from scipy.stats import zscore
panel["z_pop"] = zscore(panel["Poblacion"].fillna(panel["Poblacion"].median()))
panel["z_emp"] = zscore(panel["Empresas"].fillna(panel["Empresas"].median()))
panel["nivel_desarrollo_regional"] = (panel["z_pop"] + panel["z_emp"]) / 2.0

panel.sort_values("ventas_total", ascending=False).head()


## 7) EDA: correlaciones y dispersión

In [ ]:

num_cols = ["ventas_total","operaciones","ticket_promedio","Poblacion","Empresas","intensidad_mercado","nivel_desarrollo_regional"]
corr = panel[num_cols].corr(numeric_only=True)

plt.figure()
plt.imshow(corr, interpolation='nearest')
plt.xticks(range(len(num_cols)), num_cols, rotation=45, ha="right")
plt.yticks(range(len(num_cols)), num_cols)
plt.colorbar()
plt.title("Matriz de correlaciones")
plt.tight_layout()
plt.show()

plt.figure()
plt.scatter(panel["Empresas"], panel["ventas_total"])
plt.xlabel("Empresas (proxy actividad)")
plt.ylabel("Ventas totales")
plt.title("Ventas vs. Empresas por departamento")
plt.tight_layout()
plt.show()


## 8) Mapa coroplético: intensidad de mercado

In [ ]:

gdf_deptos["Departamento"] = gdf_deptos["Departamento"].str.upper()
gdf_deptos["Departamento"] = (gdf_deptos["Departamento"]
                              .str.replace("RIO NEGRO","RÍO NEGRO", regex=False)
                              .str.replace("PAYSANDU","PAYSANDÚ", regex=False)
                              .str.replace("TACUAREMBO","TACUAREMBÓ", regex=False)
                              .str.replace("SAN JOSE","SAN JOSÉ", regex=False))

gdf_panel = gdf_deptos.merge(panel, on="Departamento", how="left")

ax = gdf_panel.plot(column="intensidad_mercado",
                    legend=True, figsize=(7,7),
                    scheme="Quantiles", k=5, linewidth=0.5, edgecolor="black")
ax.set_title("Intensidad de mercado (ventas per cápita) por departamento")
ax.axis("off")
plt.show()


## 9) Exportar CSV enriquecido

In [ ]:

output_csv = "ventas_enriquecidas_uy.csv"
panel.to_csv(output_csv, index=False, encoding="utf-8")
print(f"Archivo exportado: {output_csv}")



## Insights (completar tras ejecutar)
- Departamentos líderes por **ventas per cápita**.
- Relación **Ventas ~ Empresas** (fuerza de correlación).
- Cambios de ranking usando **nivel de desarrollo regional**.
- Recomendaciones: productos, promociones, cobertura.
